In [1]:
from time import time
import pandas as pd

import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout

from util import make_w2v_embeddings
from util import split_and_zero_padding
from util import ManDist

In [26]:
'''with open('./data/train_snli.txt','r') as train_data:
    td=train_data.read()
    train_data.close()
with open('./data/train_snli.txt','w') as train_data:
    for word in td:
        if word=="0" or word=="1":
           word=word.replace(word, word+"\n")
    train_data.seek(0)
    train_data.write(td)
    train_data.close()  '''     

In [17]:
train_data

<_io.BufferedRandom name='./data/train_snli.txt'>

In [27]:
''''import pandas
dataframe=pandas.read_csv("./data/train_snli.txt",delimiter="\t")
dataframe.to_csv("./data/train_snli.csv", encoding='utf-8', index=False)'''

In [2]:
df = pd.read_csv (r'./data/final.csv')

In [3]:
df

,sentence1,sentence2,is_duplicate
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0
...,...,...,...
367368,A dog with a blue collar plays ball outside.,a dog is outside,1
367369,Four dirty and barefooted children.,four children have dirty feet.,1
367370,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',0
367371,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,0


In [4]:
target=df.is_duplicate
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train.to_csv("./data/train_s.csv", encoding='utf-8', index=False)
test.to_csv("./data/test_s.csv", encoding='utf-8', index=False)

In [5]:
train

,sentence1,sentence2,is_duplicate
11857,Three men work quickly preparing food in the k...,The men are sitting outside.,0
336922,A boy in a white shirt walks in the ocean.,A person is getting wet.,1
270540,People trying out foods from a local bakery at...,The people are drinkign,0
22499,A man is videotaping another man who is beginn...,A man videotaping another man,1
240629,Two men working on digging a ditch.,Men are working outside.,1
...,...,...,...
367334,A man in a blue shirt locks a blue shutter nex...,The water in the fountain had all dried up.,0
43417,Young girl in pink coat walking beside man dre...,A father pushes his young daughter on a tire s...,0
57307,"Two dogs on wet sand, each biting the same stick.",Two dogs play in the dirt.,0
280583,Three girls pretend to be rock musicans in a p...,The girls play together,1


In [6]:
embedding_dim = 300
max_seq_length = 20
use_w2v = True

In [8]:
train, embeddings = make_w2v_embeddings(train, embedding_dim=embedding_dim, empty_w2v=not use_w2v)

Loading word2vec model(it may takes 2-3 mins) ...
99,000 sentences embedded.
305,000 sentences embedded.
21,000 sentences embedded.
165,000 sentences embedded.
216,000 sentences embedded.
152,000 sentences embedded.
322,000 sentences embedded.
42,000 sentences embedded.
355,000 sentences embedded.
101,000 sentences embedded.
102,000 sentences embedded.
200,000 sentences embedded.
90,000 sentences embedded.
318,000 sentences embedded.
357,000 sentences embedded.
175,000 sentences embedded.
39,000 sentences embedded.
327,000 sentences embedded.
147,000 sentences embedded.
218,000 sentences embedded.
97,000 sentences embedded.
261,000 sentences embedded.
340,000 sentences embedded.
237,000 sentences embedded.
291,000 sentences embedded.
7,000 sentences embedded.
283,000 sentences embedded.
300,000 sentences embedded.
110,000 sentences embedded.
8,000 sentences embedded.
319,000 sentences embedded.
111,000 sentences embedded.
155,000 sentences embedded.
251,000 sentences embedded.
54,000 s

108,000 sentences embedded.
137,000 sentences embedded.
173,000 sentences embedded.
207,000 sentences embedded.


In [9]:
# Split to train validation
validation_size = int(len(train) * 0.1)
training_size = len(train) - validation_size

In [12]:
X = train[['sentence1', 'sentence2']]
Y = train['is_duplicate']

In [13]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

In [14]:
X_train = split_and_zero_padding(X_train, max_seq_length)
X_validation = split_and_zero_padding(X_validation, max_seq_length)

In [15]:
# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

In [16]:
# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [102]:
# Model variables
cpus = 2
batch_size = 1024 * cpus
n_epoch = 50
n_hidden = 100

In [110]:
# Define the shared model
x = Sequential()
#x.add(Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_shape=(max_seq_length,), trainable=False)) Requires GPU support

In [118]:
# CNN
x.add(Conv1D(250, kernel_size=5, activation='relu'))
x.add(GlobalMaxPool1D())
x.add(Dense(250, activation='relu'))
x.add(Dropout(0.3))
x.add(Dense(5, activation='sigmoid'))
#x.add(Dense(2, activation='tanh'))

In [117]:
# LSTM
x.add(LSTM((n_hidden),return_sequences=False))

In [113]:
shared_model = x

In [114]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

In [115]:
# Pack it all up into a Manhattan Distance model
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

ValueError: Input 0 of layer sequential_10 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 20]